This notebook provides recipes for loading and saving data from external sources.

# Local file system

## Uploading files from your local file system

`files.upload` returns a dictionary of the files which were uploaded.
The dictionary is keyed by the file name, the value is the data which was uploaded.

In [1]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [2]:
!cd sample_data && ls -l && cat README.md

total 55504
-r-xr-xr-x 1 root root     1697 Jan  1  2000 anscombe.json
-rw-r--r-- 1 root root   301141 Feb 19 17:17 california_housing_test.csv
-rw-r--r-- 1 root root  1706430 Feb 19 17:17 california_housing_train.csv
-rw-r--r-- 1 root root 18289443 Feb 19 17:17 mnist_test.csv
-rw-r--r-- 1 root root 36523880 Feb 19 17:17 mnist_train_small.csv
-r-xr-xr-x 1 root root      903 Jan  1  2000 README.md
This directory includes a few sample datasets to get you started.

* `california_housing_data*.csv` is California housing data from the 1990 US
  Census; more information is available at:
  https://developers.google.com/machine-learning/crash-course/california-housing-data-description

* `mnist_*.csv` is a small sample of the [MNIST
  database](https://en.wikipedia.org/wiki/MNIST_database), which is described
  at: http://yann.lecun.com/exdb/mnist/

* `anscombe.json` contains a copy of [Anscombe's
 quartet](https://en.wikipedia.org/wiki/Anscombe%27s_quartet);
  it was originally described in



## Downloading files to your local file system

`files.download` will invoke a browser download of the file to the user's local computer.


In [0]:
from google.colab import files

with open('example.txt', 'w') as f:
  f.write('some content')

files.download('example.txt')

# Google Drive

You can access files in Drive using the [native REST API](https://developers.google.com/drive/v3/web/about-sdk) or a wrapper like [PyDrive](https://googledrive.github.io/PyDrive/docs/build/html/index.html). We'll describe each, starting with PyDrive.

## PyDrive

The example below shows 1) authentication, 2) file upload, and 3) file download. More examples are available in the [PyDrive documentation](https://googledrive.github.io/PyDrive/docs/build/html/index.html)

In [4]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# PyDrive reference:
# https://googledrive.github.io/PyDrive/docs/build/html/index.html

# 2. Create & upload a file text file.
uploaded = drive.CreateFile({'title': 'Sample upload.txt'})
uploaded.SetContentString('Sample upload file content')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

# 3. Load a file by ID and print its contents.
downloaded = drive.CreateFile({'id': uploaded.get('id')})
print('Downloaded content "{}"'.format(downloaded.GetContentString()))

    100% |████████████████████████████████| 993kB 18.4MB/s 
Uploaded file with ID 1TVvCtWOeZjGULDeWd2EbfsyhqXayMeU1
Downloaded content "Sample upload file content"


## Drive REST API

The first step is to authenticate.

In [0]:
from google.colab import auth
auth.authenticate_user()

Now we can construct a Drive API client.

In [0]:
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

With the client created, we can use any of the functions in the [Google Drive API reference](https://developers.google.com/drive/v3/reference/). Examples follow.


## Creating a new Drive file with data from Python

In [7]:
# Create a local file to upload.
with open('/tmp/to_upload.txt', 'w') as f:
  f.write('my sample file')

print('/tmp/to_upload.txt contains:')
!cat /tmp/to_upload.txt

/tmp/to_upload.txt contains:
my sample file

In [9]:
# Upload the file to Drive. See:
#
# https://developers.google.com/drive/v3/reference/files/create
# https://developers.google.com/drive/v3/web/manage-uploads
from googleapiclient.http import MediaFileUpload

file_metadata = {
  'name': 'Sample file',
  'mimeType': 'text/plain'
}
media = MediaFileUpload('/tmp/to_upload.txt', 
                        mimetype='text/plain',
                        resumable=True)
created = drive_service.files().create(body=file_metadata,
                                       media_body=media,
                                       fields='id').execute()
print('File ID: {}'.format(created.get('id')))

File ID: 1F-3f5PnEXm5yhYCvQV-oNtVHeNP_VvUC


After executing the cell above, a new file named 'Sample file' will appear in your [drive.google.com](https://drive.google.com/) file list. Your file ID will differ since you will have created a new, distinct file from the example above.

## Downloading data from a Drive file into Python

In [12]:
# Download the file we just uploaded.
#
# Replace the assignment below with your file ID
# to download a different file.
#
# A file ID looks like: 1uBtlaggVyWshwcyP6kEI-y_W3P8D26sz
file_id = '1F-3f5PnEXm5yhYCvQV-oNtVHeNP_VvUC'

import io
from googleapiclient.http import MediaIoBaseDownload

request = drive_service.files().get_media(fileId=file_id)
downloaded = io.BytesIO()
downloader = MediaIoBaseDownload(downloaded, request)
done = False
while done is False:
  # _ is a placeholder for a progress object that we ignore.
  # (Our file is small, so we skip reporting progress.)
  _, done = downloader.next_chunk()

downloaded.seek(0)
print('Downloaded file contents are: {}'.format(downloaded.read()))

Downloaded file contents are: b'my sample file'


# Google Sheets

Our examples below will use the existing open-source [gspread](https://github.com/burnash/gspread) library for interacting with Sheets.

First, we'll install the package using `pip`.

In [0]:
!pip install --upgrade -q gspread

Next, we'll import the library, authenticate, and create the interface to sheets.

In [0]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

Below is a small set of gspread examples. Additional examples are shown on the [gspread Github page](https://github.com/burnash/gspread#more-examples).

## Creating a new sheet with data from Python

In [0]:
sh = gc.create('A new spreadsheet')

After executing the cell above, a new spreadsheet will be shown in your sheets list on [sheets.google.com](http://sheets.google.com/).

In [17]:
# Open our new sheet and add some data.
worksheet = gc.open('A new spreadsheet').sheet1

cell_list = worksheet.range('A1:C2')

import random
for cell in cell_list:
  cell.value = random.randint(1, 10)

worksheet.update_cells(cell_list)

{'spreadsheetId': '1RRREv5s2FbSbBLblE5eTTuFfI3J1yEAWHd2micXX0ck',
 'updatedCells': 6,
 'updatedColumns': 3,
 'updatedRange': 'Sheet1!A1:C2',
 'updatedRows': 2}

After executing the cell above, the sheet will be populated with random numbers in the assigned range.

## Downloading data from a sheet into Python as a Pandas DataFrame

We'll read back to the data that we inserted above and convert the result into a [Pandas DataFrame](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html).

(The data you observe will differ since the contents of each cell is a random number.)

In [18]:
# Open our new sheet and read some data.
worksheet = gc.open('A new spreadsheet').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
print(rows)

# Convert to a DataFrame and render.
import pandas as pd
pd.DataFrame.from_records(rows)

[['4', '3', '5'], ['6', '7', '8']]


,0,1,2
0,4,3,5
1,6,7,8


# Google Cloud Storage (GCS)

We'll start by authenticating to GCS and creating the service client.

In [0]:
from google.colab import auth
auth.authenticate_user()

## Upload a file from Python to a GCS bucket

We'll start by creating the sample file to be uploaded.

In [20]:
# Create a local file to upload.
with open('/tmp/to_upload.txt', 'w') as f:
  f.write('my sample file')

print('/tmp/to_upload.txt contains:')
!cat /tmp/to_upload.txt

/tmp/to_upload.txt contains:
my sample file

Next, we'll upload the file using the `gsutil` command, which is included by default on Colab backends.

In [0]:
# First, we need to set our project. Replace the assignment below
# with your project ID.
project_id = 'rosy-sunspot-232812'

In [23]:
!gcloud config set project {project_id}

Updated property [core/project].


In [24]:
import uuid

# Make a unique bucket to which we'll upload the file.
# (GCS buckets are part of a single global namespace.)
bucket_name = 'colab-sample-bucket-' + str(uuid.uuid1())

# Full reference: https://cloud.google.com/storage/docs/gsutil/commands/mb
!gsutil mb gs://{bucket_name}

Creating gs://colab-sample-bucket-addb36c0-39b5-11e9-8f62-0242ac1c0002/...


In [25]:
# Copy the file to our new bucket.
# Full reference: https://cloud.google.com/storage/docs/gsutil/commands/cp
!gsutil cp /tmp/to_upload.txt gs://{bucket_name}/

Copying file:///tmp/to_upload.txt [Content-Type=text/plain]...
/ [1 files][   14.0 B/   14.0 B]                                                
Operation completed over 1 objects/14.0 B.                                       


In [26]:
# Finally, dump the contents of our newly copied file to make sure everything worked.
!gsutil cat gs://{bucket_name}/to_upload.txt

my sample file

### Using Python

This section demonstrates how to upload files using the native Python API rather than `gsutil`.

This snippet is based on [a larger example](https://github.com/GoogleCloudPlatform/storage-file-transfer-json-python/blob/master/chunked_transfer.py) with additional uses of the API.

In [0]:
# The first step is to create a bucket in your cloud project.
#
# Replace the assignment below with your cloud project ID.
#
# For details on cloud projects, see:
# https://cloud.google.com/resource-manager/docs/creating-managing-projects
project_id = 'rosy-sunspot-232812'

In [28]:
# Authenticate to GCS.
from google.colab import auth
auth.authenticate_user()

# Create the service client.
from googleapiclient.discovery import build
gcs_service = build('storage', 'v1')

# Generate a random bucket name to which we'll upload the file.
import uuid
bucket_name = 'colab-sample-bucket' + str(uuid.uuid1())

body = {
  'name': bucket_name,
  # For a full list of locations, see:
  # https://cloud.google.com/storage/docs/bucket-locations
  'location': 'us',
}
gcs_service.buckets().insert(project=project_id, body=body).execute()
print('Done')

Done


The cell below uploads the file to our newly created bucket.

In [29]:
from googleapiclient.http import MediaFileUpload

media = MediaFileUpload('/tmp/to_upload.txt', 
                        mimetype='text/plain',
                        resumable=True)

request = gcs_service.objects().insert(bucket=bucket_name, 
                                       name='to_upload.txt',
                                       media_body=media)

response = None
while response is None:
  # _ is a placeholder for a progress object that we ignore.
  # (Our file is small, so we skip reporting progress.)
  _, response = request.next_chunk()

print('Upload complete')

Upload complete


Once the upload has finished, the data will appear in the cloud console storage browser for your project:

https://console.cloud.google.com/storage/browser?project=YOUR_PROJECT_ID_HERE

## Downloading a file from GCS to Python

Next, we'll download the file we just uploaded in the example above. It's as simple as reversing the order in the `gsutil cp` command.

In [30]:
# Download the file.
!gsutil cp gs://{bucket_name}/to_upload.txt /tmp/gsutil_download.txt
  
# Print the result to make sure the transfer worked.
!cat /tmp/gsutil_download.txt

Copying gs://colab-sample-bucketd1f2b1aa-39b5-11e9-8f62-0242ac1c0002/to_upload.txt...
/ [1 files][   14.0 B/   14.0 B]                                                
Operation completed over 1 objects/14.0 B.                                       
my sample file

### Using Python

We repeat the download example above using the native Python API.

In [31]:
# Authenticate to GCS.
from google.colab import auth
auth.authenticate_user()

# Create the service client.
from googleapiclient.discovery import build
gcs_service = build('storage', 'v1')

from apiclient.http import MediaIoBaseDownload

with open('/tmp/downloaded_from_gcs.txt', 'wb') as f:
  request = gcs_service.objects().get_media(bucket=bucket_name,
                                            object='to_upload.txt')
  media = MediaIoBaseDownload(f, request)

  done = False
  while not done:
    # _ is a placeholder for a progress object that we ignore.
    # (Our file is small, so we skip reporting progress.)
    _, done = media.next_chunk()

print('Download complete')

Download complete


In [32]:
# Inspect the file we downloaded to /tmp
!cat /tmp/downloaded_from_gcs.txt

my sample file